In [ ]:
import os
import re
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
import seaborn as sns

from pathlib import Path

sns.set_style("whitegrid")
plt.rcParams['font.family'] = 'Malgun Gothic'   # Windows 기본 한글 폰트
plt.rcParams['axes.unicode_minus'] = False      # 마이너스 깨짐 방지


In [ ]:
# =====================================================
# 파일 경로 설정 + CSV 로드
# =====================================================

def find_project_root() -> Path:
    p = Path.cwd()

    for parent in [p] + list(p.parents):
        if (parent / "data").exists() and (parent / "notebooks").exists():
            return parent

    return p

def latest_versioned_csv(folder: Path, base_name: str) -> Path | None:
    """
    base_name_v{n}.csv 중 최신 버전 반환
    """
    pattern = re.compile(rf"^{re.escape(base_name)}_v(\d+)\.csv$")
    best_v, best_path = None, None

    for f in folder.glob(f"{base_name}_v*.csv"):
        m = pattern.match(f.name)

        if m:
            v = int(m.group(1))

            if best_v is None or v > best_v:
                best_v, best_path = v, f

    return best_path

PROJECT_ROOT = find_project_root()

CLEAN_DIR = PROJECT_ROOT / "data" / "processed"

# 최신 버전 우선, 없으면 v1
channels_path = latest_versioned_csv(CLEAN_DIR, "channels_clean")
if channels_path is None:
    channels_path = CLEAN_DIR / "channels_clean_v1.csv"

trending_path = latest_versioned_csv(CLEAN_DIR, "trending_videos_clean")
if trending_path is None:
    trending_path = CLEAN_DIR / "youtube_trending_videos_clean_v1.csv"

EDA_FIG_DIR = PROJECT_ROOT / "figures" / "eda"
EDA_FIG_DIR.mkdir(parents=True, exist_ok=True)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("channels_path:", channels_path)
print("trending_path:", trending_path)
print("EDA figures will be saved to:", EDA_FIG_DIR)

if not channels_path.exists():
    raise FileNotFoundError(f"채널 clean 파일이 없습니다: {channels_path}")

if not Path(trending_path).exists():
    raise FileNotFoundError(f"트렌딩 clean 파일이 없습니다: {trending_path}")

yt_channels_df = pd.read_csv(channels_path, low_memory=False)
yt_trending_df = pd.read_csv(trending_path, low_memory=False)

print(f"채널 데이터 로딩 완료: {yt_channels_df.shape}")
print(f"트렌딩 데이터 로딩 완료: {yt_trending_df.shape}")
display(yt_trending_df.head(3))


In [ ]:
# =====================================================
# 채널/트렌딩 데이터 병합
# =====================================================

merged_df = pd.merge(
    yt_trending_df,
    yt_channels_df,
    left_on="channelId",
    right_on="channel_id",
    how="left"
)

print(f"병합 완료: {merged_df.shape}")


In [ ]:
# =====================================================
# 기본 정보 확인
# =====================================================

print("=== 채널 데이터 정보 ===")
display(yt_channels_df.info())
display(yt_channels_df.head())

print("=== 트렌딩 데이터 정보 ===")
display(yt_trending_df.info())
display(yt_trending_df.head())

print("=== 병합 데이터 정보 ===")
display(merged_df.info())
display(merged_df.head())


In [ ]:
# =====================================================
# 결측치 확인
# =====================================================

print("=== 채널 결측치 ===")
display(yt_channels_df.isna().sum())

print("=== 트렌딩 결측치 ===")
display(yt_trending_df.isna().sum())

print("=== 병합 결측치 ===")
display(merged_df.isna().sum())


In [ ]:
# =====================================================
# 채널별 트렌딩 영상 수 계산 및 추가
# =====================================================

# channelId → channel_id 통일
yt_trending_df = yt_trending_df.rename(columns={"channelId": "channel_id"})

# 채널별 트렌딩 영상 수
channel_trending_counts = (
    yt_trending_df
    .groupby("channel_id")["video_id"]
    .nunique()
    .reset_index(name="channel_trending_video_count")
)

# 채널 데이터에 병합
yt_channels_df = yt_channels_df.merge(
    channel_trending_counts,
    on="channel_id",
    how="left"
)

yt_channels_df["channel_trending_video_count"] = (
    yt_channels_df["channel_trending_video_count"].fillna(0)
)


In [ ]:
# =====================================================
# 채널 규모 분포 시각화
# =====================================================

# 구독자 수 분포
plt.figure(figsize=(8,5))
sns.histplot(yt_channels_df["subscriber_count"], bins=50, log_scale=True)
plt.title("구독자 수 분포 (로그 스케일)")
plt.xlabel("subscriber_count")
plt.show()

# 채널 영상 수 분포
plt.figure(figsize=(8,5))
sns.histplot(yt_channels_df["video_count"], bins=50, log_scale=True)
plt.title("채널 영상 수 분포 (로그 스케일)")
plt.xlabel("video_count")
plt.show()

# 채널별 트렌딩 영상 수 분포
if "channel_trending_video_count" in yt_channels_df.columns:
    plt.figure(figsize=(8,5))
    sns.histplot(yt_channels_df["channel_trending_video_count"], bins=30)
    plt.title("채널별 트렌딩 영상 수")
    plt.show()


In [ ]:
# =====================================================
# 채널 성과 비율 확인
# =====================================================

if "channel_trending_ratio" in yt_channels_df.columns:
    plt.figure(figsize=(8,5))
    sns.histplot(yt_channels_df["channel_trending_ratio"], bins=30)
    plt.title("채널별 트렌딩 비율")
    plt.show()


In [ ]:
# =====================================================
# 카테고리·국가별 요약
# =====================================================

cat_summary = yt_channels_df.groupby("category").agg(
    avg_subscribers=("subscriber_count", "mean"),
    avg_views=("view_count", "mean"),
    avg_trending_ratio=("channel_trending_ratio", "mean") if "channel_trending_ratio" in yt_channels_df.columns else ("subscriber_count","mean"),
    count_channels=("channel_id", "count")
).sort_values("avg_trending_ratio", ascending=False)

print("카테고리별 요약 (상위 10):")
display(cat_summary.head(10))

country_summary = yt_channels_df.groupby("country").agg(
    avg_subscribers=("subscriber_count", "mean"),
    avg_views=("view_count", "mean"),
    avg_trending_ratio=("channel_trending_ratio", "mean") if "channel_trending_ratio" in yt_channels_df.columns else ("subscriber_count","mean"),
    count_channels=("channel_id", "count")
).sort_values("avg_trending_ratio", ascending=False)

print("국가별 요약 (상위 10):")
display(country_summary.head(10))


In [ ]:
# =====================================================
# 트렌딩 데이터 기반 추가 분석
# =====================================================

channel_perf = merged_df.groupby("channelId").agg(
    total_trending_videos=("video_id", "nunique"),
    total_views=("view_count_x", "sum") if "view_count_x" in merged_df.columns else ("view_count","sum"),
    total_likes=("likes", "sum"),
    total_comments=("comment_count", "sum"),
    avg_engagement=("engagement_score", "mean") if "engagement_score" in merged_df.columns else ("likes","mean")
).sort_values("total_trending_videos", ascending=False)

print("채널별 트렌딩 성과 (상위 10):")
display(channel_perf.head(10))


In [ ]:
# =====================================================
# 시계열 분석 (최근 트렌딩 추세)
# =====================================================

merged_df["trending_date_only"] = pd.to_datetime(merged_df["trending_date"]).dt.date
recent_trending = merged_df.groupby("trending_date_only").size()

plt.figure(figsize=(12,5))
recent_trending.plot(kind="line")
plt.title("날짜별 트렌딩 영상 수")
plt.ylabel("트렌딩 영상 수")
plt.xlabel("날짜")
plt.show()

In [ ]:
# =====================================================
# 영상 단위 성과 지표 생성
# =====================================================

yt_trending_df = yt_trending_df.copy()

# 조회수 0 → NA 처리
yt_trending_df["view_count_safe"] = yt_trending_df["view_count"].replace(0, pd.NA)

# 각종 반응률
yt_trending_df["engagement_rate"] = (
    yt_trending_df["likes"] + yt_trending_df["comment_count"]
) / yt_trending_df["view_count_safe"]
yt_trending_df["like_rate"] = yt_trending_df["likes"] / yt_trending_df["view_count_safe"]
yt_trending_df["comment_rate"] = yt_trending_df["comment_count"] / yt_trending_df["view_count_safe"]

yt_trending_df[
    ["video_id", "view_count", "likes", "comment_count", 
    "engagement_rate", "like_rate", "comment_rate"]
].head()


In [ ]:
# =====================================================
# video_id 기준 영상 단위 성과 테이블
# =====================================================

video_metrics_df = (
    yt_trending_df
    .groupby("video_id")
    .agg(
        
        # 영상 기본 정보
        channel_id=("channelId", "first") if "channelId" in yt_trending_df.columns else ("channel_id", "first"),
        title=("title", "first"),
        country=("country", "first"),

        # 트렌딩 관련
        trending_days=("trending_days", "max"),   # 트렌딩 유지 일수
        max_views=("view_count", "max"),   # 트렌딩 기간 중 최대 조회수

        # 영상 단위 반응 지표 (평균)
        avg_engagement_rate=("engagement_rate", "mean"),
        avg_like_rate=("like_rate", "mean"),
        avg_comment_rate=("comment_rate", "mean")
    )
    .reset_index()
)

video_metrics_df.head()


In [ ]:
# =====================================================
# 채널 구독자 수 병합
# =====================================================

video_metrics_with_channel = video_metrics_df.merge(
    yt_channels_df[["channel_id", "subscriber_count"]],
    on="channel_id",
    how="left"
)

video_metrics_with_channel[["video_id", "subscriber_count", "avg_engagement_rate"]].head()


In [ ]:
# =====================================================
# 구독자 규모 그룹화 (소형/중형/대형)
# =====================================================

tmp = video_metrics_with_channel.dropna(subset=["subscriber_count", "avg_engagement_rate"]).copy()

tmp["subscriber_group"] = pd.qcut(
    tmp["subscriber_count"],
    q=3,   # 3개 구간 → 소형 / 중형 / 대형
    labels=["소형", "중형", "대형"]
)

tmp[["video_id", "subscriber_count", "subscriber_group", "avg_engagement_rate"]].head()


In [ ]:
# =====================================================
# 구독자 규모별 영상 참여도 분석
# =====================================================

subscriber_group_summary = (
    tmp
    .groupby("subscriber_group")
    .agg(
        avg_engagement=("avg_engagement_rate", "mean"),
        median_engagement=("avg_engagement_rate", "median"),
        n_videos=("video_id", "nunique")
    )
    .reset_index()
)

print(subscriber_group_summary)

plt.figure(figsize=(8,5))

sns.boxplot(
    data=tmp,
    x="subscriber_group",
    y="avg_engagement_rate"
)

plt.title("구독자 규모별 영상 평균 참여도 분포")
plt.xlabel("구독자 규모 (소형 / 중형 / 대형)")
plt.ylabel("영상 단위 평균 참여도 (engagement_rate)")
plt.show()


In [ ]:
# =====================================================
# [EDA Figure 1] 트렌딩 유지기간(trending_days) 분포
# =====================================================

# video_id 기준 trending_days 계산
trending_days_df = (
    yt_trending_df
    .groupby("video_id")["trending_date"]
    .nunique()
    .reset_index(name="trending_days")
)

plt.figure(figsize=(8, 5))
sns.histplot(trending_days_df["trending_days"], bins=50)
plt.yscale("log")

plt.title("트렌딩 유지기간 분포 (trending_days)")
plt.xlabel("Trending Duration (days)")
plt.ylabel("Count (log scale)")

plt.tight_layout()
plt.savefig(EDA_FIG_DIR / "fig1_trending_days_distribution.png", dpi=300)
plt.show()


In [ ]:
# =====================================================
# [EDA Figure 2] 채널별 트렌딩 영상 수 분포
# =====================================================

plt.figure(figsize=(8, 5))
sns.histplot(
    channel_trending_counts["channel_trending_video_count"],
    bins=50
)
plt.yscale("log")

plt.title("채널별 트렌딩 영상 수 분포")
plt.xlabel("Number of Trending Videos per Channel")
plt.ylabel("Count (log scale)")

plt.tight_layout()
plt.savefig(EDA_FIG_DIR / "fig2_channel_trending_video_distribution.png", dpi=300)
plt.show()
